In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.max_columns = 100

import seaborn as sns
import pandas_profiling as pdp
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

from IPython.display import display
import collections
import re
import feather
import codecs

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold

In [3]:
train=pd.read_csv('../input/train.csv')
test=pd.read_csv('../input/test.csv')
df_all=pd.concat([train,test],axis=0,sort=False).reset_index(drop=True)
df_all.columns=['id','rent','address','access','floor_info','old','direction','square','floor','bath','kitchen','broadcast','facility','parking','neighbors','structure','period']


train_feat1=pd.read_feather('feature1/data/train_feat1.ftr')
test_feat1=pd.read_feather('feature1/data/test_feat1.ftr')

## 目的変数外れ値・異常値の除去
RMSEは外れ値の影響を大きく受けるため

In [4]:
# 外れ値、異常データを削除

def drop_train_outlier(df_train):
    train_feat1=pd.read_feather('feature1/data/train_feat1.ftr')
    train=df_train[df_train['id']!=20428][df_train['id']!=20232][df_train['id']!=20927]
    train_feat1=train_feat1[train_feat1['id']!=20428][train_feat1['id']!=20232][train_feat1['id']!=20927]

# 面積に対する外れ値を取得
    index1=train_feat1.query("rent>1000000 & square_num<100").index
    index2=train_feat1.query("rent>1500000").index
    index3=train_feat1.query("square_num>400").index
    index4=train_feat1.query('rent>100000 & square_num<10').index
    
# 築年数に対する外れ値を取得
    index5=train_feat1.query("old_num>1000").index
    
    drop_index=np.concatenate([index1,index2,index3,index4,index5])
    
    train_drop=train.drop(drop_index).reset_index(drop=True)
    train_feat1_drop=train_feat1.drop(drop_index).reset_index(drop=True)
    
    return train_drop

# テストデータ中の外れ値を補正
def modify_test_outlier(df_test):
    test_feat1=pd.read_feather('feature1/data/test_feat1.ftr')

    df_test.loc[test_feat1.query('old_num>1000').index,'築年数']='11年4ヶ月'
    return df_test
    
train_drop=drop_train_outlier(train)
test_mod=modify_test_outlier(test)

train_index=len(train_drop)
test_index=len(test_mod)

df_all=pd.concat([train_drop,test_mod],axis=0,sort=False).reset_index(drop=True)
df_all.columns=['id','rent','address','access','floor_info','old','direction','square','floor','bath','kitchen','broadcast','facility','parking','neighbors','structure','period']

train_feat1_drop=pd.merge(train_drop.loc[:,['id']],train_feat1,on='id',how='left')
test_feat1_mod=modify_test_outlier(test_feat1)

df_all_feat1=pd.concat([train_feat1_drop,test_feat1_mod],axis=0,sort=False).reset_index(drop=True)

## 間取り情報に部屋数、配置を追加

In [5]:
def add_floor_detail(df):
    # LK をDKのタイポとし、DKに変換
    df=df.replace('1LK+S(納戸)','1DK+S(納戸)')
    df=df.replace('1LK','1DK')
    floor_info=pd.read_csv('../input/floor_info.csv',encoding="utf_8")
    df_merge=pd.merge(df,floor_info,on='floor_info',how='left')
    return df_merge

In [6]:
df_all=add_floor_detail(df_all)

##  部屋総数を追加
1K=1R 1DK=1.5R  1LDK=2Rとして換算

In [7]:
def add_room_number(df):
    # ルームごとのフロア面積の平均を求める
    floor_square=pd.DataFrame()
    floor_square['floor_info']=df_all['floor_info']
    floor_square['square']=df_all_feat1['square_num']
    floor_mean=floor_square.groupby('floor_info')['square'].mean().reset_index()

    R_value=floor_mean.query("floor_info=='1R'")['square'].values
    K_value=floor_mean.query("floor_info=='1K'")['square'].values-R_value
    DK_value=floor_mean.query("floor_info=='1DK'")['square'].values-R_value
    LDK_value=floor_mean.query("floor_info=='1LDK'")['square'].values-R_value

    S_ofK_value=floor_mean.query("floor_info=='1K+S(納戸)'")['square'].values-floor_mean.query("floor_info=='1K'")['square'].values
    S_ofDK_value=floor_mean.query("floor_info=='1DK+S(納戸)'")['square'].values-floor_mean.query("floor_info=='1DK'")['square'].values
    S_ofLDK_value=floor_mean.query("floor_info=='1LDK+S(納戸)'")['square'].values-floor_mean.query("floor_info=='1LDK'")['square'].values

    # 各間取りの1Rに対する面積比を求める
    K=K_value/R_value
    DK=DK_value/R_value
    LDK=LDK_value/R_value
    S_K=S_ofK_value/R_value
    S_DK=S_ofDK_value/R_value
    S_LDK=S_ofLDK_value/R_value
    
    room_number=df['R']+df['K']*K+df['DK']*DK+df['LDK']*LDK+(df['K']>=1)*1*S_K+(df['DK']>=1)*1*S_DK+(df['LDK']>=1)*1*S_LDK
    
    df['room_number']=room_number
    return df

In [8]:
df_all=add_room_number(df_all)

## 一部屋あたりの部屋面積を追加

In [9]:
def add_square_per_room(df):
    S_per_R=df_all_feat1['square_num']/df['room_number']
    df['Square/Room']=S_per_R
    return df

In [10]:
df_all=add_square_per_room(df_all)

## 経度と緯度をメッシュで区切ってカテゴリー変数にする

In [11]:
with codecs.open("../input/13000-17.0a/13_2018.csv", "r", "Shift-JIS", "ignore") as file:
    df_cordA = pd.read_table(file, delimiter=",")
    #display(df_cordA[:3])    
with codecs.open("../input/13000-12.0b/13_2018.csv", "r", "Shift-JIS", "ignore") as file:
    df_cordB = pd.read_table(file, delimiter=",")
    #display(df_cordB[:3])
    
cordinate=df_cordA[['市区町村名','大字・丁目名','緯度','経度']]
cordinate.columns=['address_city','address_town','longitude','latitude']
cordinate.drop_duplicates(subset=['address_city','address_town'],keep='last',inplace=True)

In [12]:
def add_lat_long(df):
# 住所から区を取得
    city_tmp=df['address'].apply(lambda x:x.split('都')[1])
    df['address_city']=city_tmp.apply(lambda x:x.split('区')[0]+'区')

    # 住所から町名を取得
    town_tmp=city_tmp.apply(lambda x:x.split('区')[1])

    def town_enc(x):
        # 番地を削除
        x=re.split(r'\d+-|ー|－+\d+',x)[0]   
        # 何丁目か分かるものと、わからないもので分類
        split_list=x.split('丁目')
        if len(split_list)==2:
            return split_list[0]+'丁目'
        else:
        # 余分な数字を削除
            town=re.split(r'\d+',split_list[0])[0]
            return town

    def int2kanji(x):
        kanji_nums={'1':'一', '2':'二', '3':'三', '4':'四', '5':'五', '6':'六', '１':'一', '２':'二', '３':'三', '４':'四', '５':'五', '６':'六', '７':'七', '８':'八', '９':'九'}
        num=re.findall(r'\d+',x)
        if len(num)==0:
            return x
        else:
            return x.replace(num[0],kanji_nums[num[0]])
    df['address_town']=town_tmp.apply(lambda x:town_enc(x))
    df['address_town']=df['address_town'].apply(lambda x:int2kanji(x))
    
    df=pd.merge(df,cordinate,on=['address_city','address_town'],how='left')
    
    return df

In [13]:
df_all=add_lat_long(df_all)

In [14]:
def add_bin_latitude(df,bins):
    return pd.cut(df['latitude'],bins).astype(str)

def add_bin_longitude(df,bins):
    return pd.cut(df['longitude'],bins).astype(str)

In [15]:
df_all['cat_latitude']=add_bin_latitude(df_all,50)
df_all['cat_longitude']=add_bin_longitude(df_all,50)

In [16]:
def add_lat_long_category(df):
    le_lat=LabelEncoder()
    df['cat_latitude'].fillna('nan',inplace=True)
    le_lat.fit(list(df['cat_latitude'].astype(str).values))
    lat_str=le_lat.transform(list(df['cat_latitude'].astype(str).values))

    le_long=LabelEncoder()
    df['cat_longitude'].fillna('nan',inplace=True)
    le_long.fit(list(df['cat_longitude'].astype(str).values))
    long_str=le_long.transform(list(df['cat_longitude'].astype(str).values))
    
    lat_long_category=[]
    for lat,long in zip(lat_str,long_str):
        lat_long_category.append(str(lat)+'_'+str(long))
        
    return lat_long_category

def mesh_encoder(df):
    mesh_le=LabelEncoder()
    mesh_le.fit(df_all['mesh_cateogry'])
    mesh_encode=mesh_le.transform(df_all['mesh_cateogry'])
    return mesh_encode

In [17]:
df_all['mesh_cateogry']=add_lat_long_category(df_all)
df_all['mesh_category_enc']=mesh_encoder(df_all)
df_all['mesh_category_enc']=df_all['mesh_category_enc'].astype('category')

## カテゴリー変数でTargetEncoding
クロスバリデーションをする際に、foldごとに作り直さないといけないので注意

In [955]:
target_cols=['floor_info','direction','direction','direction','mesh_cateogry']

def target_encode(df,target_cols):
    train_df=df[:train_index]
    test_df=df[train_index:]
    for col in target_cols:
        # 訓練データから、一時的にカテゴリー変数と目的変数のDFを作成
        data_tmp=pd.DataFrame({col:train_df[col],'target':train_df['rent']})
        # テストデータのカテゴリを訓練データの平均値で埋める
        target_mean=data_tmp.groupby(col)['target'].mean()
        test_df[col+'_target_enc']=test_df[col].map(target_mean)
        
        # 訓練データの変換後の値を格納する配列
        train_encode=np.repeat(np.nan,train_df.shape[0])
        
        kf=KFold(n_splits=4,shuffle=True,random_state=0)
        
        for idx_1,idx_2 in kf.split(train_df):
            
            # one-of-foldで各カテゴリの目的変数の平均を計算
            target_mean=data_tmp.iloc[idx_1].groupby(col)['target'].mean()
            
            # 変換後の値を一時配列に格納
            train_encode[idx_2]=train_df[col].iloc[idx_2].map(target_mean)
            
        train_df[col+'_target_enc']=train_encode
    
    return pd.concat([train_df,test_df],axis=0,sort=False).reset_index(drop=True)

In [956]:
df_all=target_encode(df_all,target_cols)

## 同じ物件がある場合は、その情報を追加

In [54]:
# 数字を半角にそろえる
def lower_num(x):
    return x.translate(str.maketrans({chr(0xFF01 + i): chr(0x21 + i) for i in range(94)}))

df_all['address']=df_all['address'].apply(lambda x:lower_num(x))

In [55]:
df_train=df_all[:train_index]
df_test=df_all[train_index:]

house_unique=df_train.groupby(['address','access','old']).agg({'id':'size','rent':'sum','rent':{'sum','mean'}}).reset_index()
house_unique.columns=['address','access','old','id_unique','rent_sum','rent_mean']

In [56]:
def add_others_train(idx):
    address=df_train.iloc[idx]['address']
    access=df_train.iloc[idx]['access']
    old=df_train.iloc[idx]['old']
    rent=df_train.iloc[idx]['rent']
    other_house=house_unique.query('old==@old & address==@address & access==@access')
    if other_house['id_unique'].values !=1:
        other_mean=other_house['rent_sum']-rent/other_house['id_unique']-1
        other_mean=other_mean.values[0]
    else:
        other_mean=0
    return other_mean

In [57]:
train_others=[]
for idx in tqdm(df_t
                rain.index):
    train_others.append(add_others_train(idx))

100%|██████████| 31461/31461 [03:09<00:00, 165.59it/s]


In [58]:
df_train=df_train.rename(columns={'ohters_rent': 'others_rent'})

In [59]:
df_train['others_rent']=train_others

In [60]:
def add_others_test(idx):
    address=df_test.iloc[idx]['address']
    access=df_test.iloc[idx]['access']
    old=df_test.iloc[idx]['old']
    other_house=house_unique.query('old==@old & address==@address & access==@access')
    other_mean=other_house['rent_mean']
    other_mean=other_mean.values
    if len(other_mean)==1:
        return other_mean[0]
    else:
        return 0

In [61]:
test_others=[]
for idx in tqdm(range(len(df_test))):
    test_others.append(add_others_test(idx))

100%|██████████| 31262/31262 [02:56<00:00, 177.01it/s]


In [62]:
df_test['others_rent']=test_others

In [63]:
df_all=pd.concat([df_train,df_test],axis=0,sort=False).reset_index(drop=True)

In [64]:
df_all[['id','others_rent']].to_csv('others_rent.csv',index=False)

In [23]:
df_all['others_rent']=pd.read_csv('others_rent.csv')['others_rent']

## =====feature3 保存=======

In [24]:
df_all

,id,rent,address,access,floor_info,old,direction,square,floor,bath,kitchen,broadcast,facility,parking,neighbors,structure,period,R,K,DK,LDK,S,room_number,Square/Room,address_city,address_town,longitude,latitude,cat_latitude,cat_longitude,mesh_cateogry,mesh_category_enc,others_rent
0,1,75000.0,東京都北区滝野川３丁目,都営三田線\t西巣鴨駅\t徒歩4分\t\t埼京線\t板橋駅\t徒歩14分\t\t都電荒川線\...,1K,9年9ヶ月,南東,20.01m2,1階／12階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄便座,ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯,インターネット対応／\tCATV／\tCSアンテナ／\tBSアンテナ,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t室内洗濯機置場／\t敷...,駐輪場\t空有,【小学校】 495m\t【大学】 461m\t【小学校】 962m\t【公園】 1103m\...,RC（鉄筋コンクリート）,2年間,1,1,0,0,0,1.454488,13.757421,北区,滝野川三丁目,35.746889,139.731307,"(139.727, 139.734]","(35.745, 35.751]",23_37,515,0.0
1,2,76000.0,東京都中央区月島３丁目,都営大江戸線\t勝どき駅\t徒歩5分\t\t有楽町線\t月島駅\t徒歩9分\t\t日比谷線\...,1R,44年10ヶ月,NaN,16.5m2,5階／10階建,専用トイレ／\tシャワー／\t温水洗浄便座,ガスコンロ／\tシステムキッチン\t／\t給湯,インターネット対応,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t室内洗濯機置場／\t敷...,駐輪場\t空有\t駐車場\t無\tバイク置き場\t無,【スーパー】 1283m,鉄骨造,2年間,1,0,0,0,0,1.000000,16.500000,中央区,月島三丁目,35.660879,139.778811,"(139.777, 139.784]","(35.657, 35.662]",30_21,715,0.0
2,3,110000.0,東京都渋谷区笹塚２丁目,京王線\t笹塚駅\t徒歩6分\t\t京王線\t代田橋駅\t徒歩7分\t\t京王線\t明大前駅...,1K,8年6ヶ月,南,22.05m2,12階／15階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄...,ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯,インターネット対応／\t光ファイバー／\tCSアンテナ／\tBSアンテナ,エアコン付\tウォークインクローゼット\tシューズボックス／\tバルコニー／\tフローリング...,"駐輪場\t空有\tバイク置き場\t空有\t駐車場\t近隣\t30,000円(税込)\t距離100m",【スーパー】 89m\t【コンビニ】 184m\t【コンビニ】 392m\t【スーパー】 492m,RC（鉄筋コンクリート）,2年間,1,1,0,0,0,1.454488,15.159977,渋谷区,笹塚二丁目,35.675199,139.666064,"(139.664, 139.671]","(35.673, 35.679]",14_24,147,54499.0
3,4,150000.0,東京都杉並区高円寺南２丁目23-2,総武線・中央線（各停）\t高円寺駅\t徒歩9分\t\t丸ノ内線(池袋－荻窪)\t新高円寺駅\...,2LDK,29年4ヶ月,南,60.48m2,3階／4階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t温水洗浄便座／\t洗面台独立,ガスコンロ／\t給湯,インターネット対応／\t光ファイバー,エアコン付\tシューズボックス／\tバルコニー／\t2面採光／\t室内洗濯機置場／\tエレベ...,駐車場\t無\t駐輪場\t無\tバイク置き場\t無,【スーパー】 225m\t【スーパー】 448m\t【スーパー】 619m\t【スーパー】 ...,RC（鉄筋コンクリート）,2年間\t※この物件は\t定期借家\tです。,2,0,0,1,0,3.726175,16.231122,杉並区,高円寺南二丁目,35.700157,139.650554,"(139.65, 139.657]","(35.695, 35.701]",12_28,85,0.0
4,5,74000.0,東京都葛飾区金町３丁目7-2,京成金町線\t京成金町駅\t徒歩5分\t\t常磐線\t金町(東京都)駅\t徒歩7分\t\t京...,2DK,31年7ヶ月,南,39.66m2,1階／2階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t追焚機能,給湯／\t独立キッチン,NaN,バルコニー／\tフローリング／\t室内洗濯機置場\t公営水道／\t下水,"駐車場\t近隣\t17,000円(税込)\t距離300m\t駐輪場\t無\tバイク置き場\t無",【スーパー】 193m\t【スーパー】 298m\t【スーパー】 660m\t【スーパー】 ...,木造,2年間,2,0,1,0,0,2.996878,13.233772,葛飾区,金町三丁目,35.770009,139.878559,"(139.875, 139.882]","(35.767, 35.773]",44_41,1093,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62718,62728,NaN,東京都豊島区上池袋４丁目,埼京線\t板橋駅\t徒歩3分\t\t都営三田線\t新板橋駅\t徒歩9分\t\t東武東上線\t...,1K,0年1ヶ月,東,25.66m2,6階／8階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄...,ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯,インターネット対応／\t光ファイバー／\tCATV／\tCSアンテナ／\tBSアンテナ／\t...,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t室内洗濯機置場／\t敷...,駐車場\t無\t駐輪場\t無\tバイク置き場\t無,【コンビニ】 204m\t【スーパー】 284m\t【スーパー】 496m\t【コンビニ】 ...,RC（鉄筋コンクリート）,2年間,1,1,0,0,0,1.454488,17.641950,豊島区,上池袋四丁目,35.740681,139.720950,"(139.72, 139.727]","(35.74, 35.745]",22_36,475,907500.0
62719,62729,NaN,東京都千代田区岩本町２丁目,都営新宿線\t岩本町駅\t徒歩2分\t\t山手線\t秋葉原駅\t徒歩4分\t\t日比谷線\t...,1R,15年6ヶ月,南,22.71m2,8階／15階建,専用バス／\t専用トイレ／\tシャワー／\t浴室乾燥機,ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t独立キッチン,インターネット対応／\t高速インターネット／\t光ファイバー／\tCATV／\tCSアンテナ...,冷房／\tエアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t24時間換気...,"駐車場\t空有\t43,200円\t駐輪場\t空有",NaN,RC（鉄筋コンクリート）,NaN,1,0,0,0,0,1.000000,22.710000,千代田区,岩本町二丁目,35.693770,139.778852,"(139.777, 139.784]","(35.69, 35.695]",30_27,721,0.0
62720,62730,NaN,東京都中野区江古田３丁目14-3,都営大江戸線\t新江古田駅\t徒歩10分\t\t西武池袋線\t江古田駅\t徒歩20分\t\t...,1LDK,1年0ヶ月,南東,45.76m2,10階／14階建,バス・トイレ別／\tシャワー／\t追焚機能／\t温水洗浄便座,ガスコンロ／\tコンロ3口／\t給湯,インターネット対応／\tCATV／\tインターネット使用料無料,エアコン付／\t床暖房\tフローリング／\t室内洗濯機置場／\t敷地内ごみ置き場\t／\tエ...,"駐車場\t空有\t32,400円(税込)\t駐輪場\t空有\tバイク置き場\t無",NaN,RC（鉄筋コンクリート）,2年間\t※この物件は\t定期借家\tです。,1,0,0,1,0,2.726175,16.785423,中野区,江古田三丁目,35.728242,139.664074,"(139.664, 139.671]","(35.723, 35.728]",1

In [67]:
df_train=df_all[:train_index]
df_test=df_all[train_index:]

feat3_cols=['id','R', 'K', 'DK', 'LDK','S', 'room_number', 'Square/Room', 'mesh_category_enc', 'others_rent']
train_feat3=df_train[feat3_cols]
test_feat3=df_test[feat3_cols]

train_feat3.to_feather('train_feat3_unique.ftr')
test_feat3.reset_index(inplace=True)
test_feat3.to_feather('test_feat3_unique.ftr')

### 今までの特徴と合わせる

In [1441]:
train_feat2=pd.read_feather('feature2/data/train_feat2.ftr')
test_feat2=pd.read_feather('feature2/data/test_feat2.ftr')

In [1442]:
train_feat3=pd.merge(train_feat3,train_feat2,on='id')
test_feat3=pd.merge(test_feat3,test_feat2,on='id')

In [1443]:
train_feat3.to_feather('train_feat3.ftr')
test_feat3.reset_index(inplace=True)
test_feat3.to_feather('test_feat3.ftr')